In [2]:
import pandas as pd
from unidecode import unidecode
import numpy as np

In [51]:
import webbrowser

def remove_accents(input_str):
    return unidecode(input_str)

def search_and_display(csv_file, search_string):
    try:
        # Read data from the CSV file
        df = pd.read_csv(csv_file)

        # Prompt the user to select a search option
        while True:
            print("Select a search option:")
            print("1. Search in a specific language")
            print("2. Search across all languages")

            choice = input("Enter the number of your choice: ")

            if choice == '1':
                # Search in a specific language
                languages = df.columns.tolist()
                while True:
                    print("Select a language:")
                    for i, lang in enumerate(languages, 1):
                        print(f"{i}. {lang}")

                    lang_choice = input("Enter the number of your choice: ")

                    if not lang_choice.isdigit() or int(lang_choice) < 1 or int(lang_choice) > len(languages):
                        print("Invalid choice. Please select a valid language.")
                    else:
                        language = languages[int(lang_choice) - 1]
                        break

                # Remove accents from the search string
                search_string = remove_accents(search_string)

                # Remove accents from the data in the selected column
                df[language] = df[language].apply(remove_accents)

                # Perform the partial string match search
                result = df[df[language].str.contains(search_string, case=False, na=False)]

                if result.empty:
                    print("No matching rows found.")
                else:
                    print("Choose an option:")
                    options = result[language].tolist()
                    for i, option in enumerate(options, 1):
                        print(f"{i}. {option}")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(options):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_option = result.iloc[choice - 1]
                        print("\nChosen Option:")
                        print(f"{chosen_option[language]}\n")
                        
                        # Get the translations in other languages
                        translations = {}
                        for col in df.columns:
                            if col != language and col != 'en':  # Exclude the selected language and 'en' column
                                translations[col] = chosen_option[col]
                        
                        # Display the translations
                        print(f"{language}: {chosen_option[language]}")
                        for lang, translation in translations.items():
                            print(f"{lang}: {translation}")

                        # Ask if the user wants to Google search the chosen word
                        google_search_choice = input("Do you want to Google search this term? (yes/no): ").strip().lower()
                        if google_search_choice == 'yes':
                            google_search_url = f"https://www.google.com/search?q={chosen_option[language]}"

            elif choice == '2':
                # Search across all languages
                search_string = remove_accents(search_string)
                matching_words = []
                matching_languages = {}
                
                for col in df.columns:
                    col_matches = df[col][df[col].str.contains(search_string, case=False, na=False)].tolist()
                    if col_matches:
                        matching_words.extend(col_matches)
                        matching_languages[col] = col_matches

                if not matching_words:
                    print("No matching words found.")
                else:
                    print("Choose an option:")
                    for i, word in enumerate(matching_words, 1):
                        print(f"{i}. {word} ({', '.join([col for col, translations in matching_languages.items() if word in translations])})")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(matching_words):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_word = matching_words[choice - 1]
                        print("\nChosen Word:")
                        print(f"{chosen_word} ({', '.join([col for col, translations in matching_languages.items() if chosen_word in translations])})")

                        print("\nTranslations in Other Languages:")
                        for lang, translations in matching_languages.items():
                            if chosen_word in translations:
                                print(f"{lang}: {translations[translations.index(chosen_word)]}")

                        # Ask if the user wants to Google search the chosen word
                        google_search_choice = input("Do you want to Google search this term? (yes/no): ").strip().lower()
                        if google_search_choice == 'yes':
                            google_search_url = f"https://www.google.com/search?q={chosen_word}"

            else:
                print("Invalid choice. Please select a valid search option.")

            # Open the Google search URL if chosen
            if google_search_choice == 'yes':
                webbrowser.open(google_search_url)
                break

    except Exception as e:
        return f"An error occurred: {str(e)}"

In [52]:
csv_file = 'dictionary_data.csv'
search_string = input("Enter search string: ")
search_and_display(csv_file, search_string)

Enter search string:  math


Select a search option:
1. Search in a specific language
2. Search across all languages


Enter the number of your choice:  1


Select a language:
1. es
2. de
3. en


Enter the number of your choice:  3


Choose an option:
1. Mathematical analysis
2. Mathematical Analysis
3. Mathematics


Enter the number of your choice:  1



Chosen Option:
Mathematical analysis

en: Mathematical analysis
es: Análisis matemático
de: Analysis


Do you want to Google search this term? (yes/no):  yes
